In [1]:
%pip install -q segmentation-models-pytorch

Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.model_selection import KFold
import os
import pandas as pd
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Subset, Dataset
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import random
from torchvision.transforms import ToTensor
from PIL import Image
import os

import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torchvision import transforms
# from torchinfo import summary
import timm
import segmentation_models_pytorch as smp
import wandb
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
images_path = "/kaggle/input/bkai-igh-neopolyp/train/train/"
image_path = []
TRAIN_DIR = '/kaggle/input/bkai-igh-neopolyp/train/train'
for root, dirs, files in os.walk(TRAIN_DIR):
    for file in files:
        path = os.path.join(root,file)
        image_path.append(path)
        
len(image_path)

1000

In [4]:
mask_path = []
TRAIN_MASK_DIR = '/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt'
for root, dirs, files in os.walk(TRAIN_MASK_DIR):
    for file in files:
        path = os.path.join(root,file)
        mask_path.append(path)
        
len(mask_path)

1000

In [5]:
class TrainDataset(Dataset):
    def __init__(self, img_dir, label_dir, resize=None, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.resize = resize
        self.transform = transform
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)
    
    def read_mask(self, mask_path):
        image = cv2.imread(mask_path)
        image = cv2.resize(image, self.resize, interpolation=cv2.INTER_AREA)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        lower_red1 = np.array([0, 100, 20])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160,100,20])
        upper_red2 = np.array([179,255,255])
        
        lower_mask_red = cv2.inRange(image, lower_red1, upper_red1)
        upper_mask_red = cv2.inRange(image, lower_red2, upper_red2)
        
        red_mask = lower_mask_red + upper_mask_red
        red_mask[red_mask != 0] = 1

        green_mask = cv2.inRange(image, (36, 25, 25), (70, 255, 255))
        green_mask[green_mask != 0] = 2

        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = np.expand_dims(full_mask, axis=-1) 
        full_mask = full_mask.astype(np.uint8)
        
        return full_mask

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx])
        image = cv2.imread(img_path)  #  BGR
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB
        label = self.read_mask(label_path)  
        image = cv2.resize(image, self.resize, interpolation=cv2.INTER_AREA)
        if self.transform:
            image = self.transform(image)
            
        return image, label
    
class TestDataset(Dataset):
    def __init__(self, img_dir="path/to/data", resize = None, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.resize = resize
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)
    

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, channels = image.shape
        image = cv2.resize(image, self.resize, interpolation=cv2.INTER_AREA)
        name =  os.path.basename(img_path)
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]
        return image, height, width, name[:-5]

In [6]:
class AugmentDataset(Dataset):
    def __init__(self, dataset, transform=None, length_multiplier=1):
        self.dataset = dataset
        self.transform = transform
        self.length_multiplier = length_multiplier
        
    def __getitem__(self, idx):
        actual_idx = idx % len(self.dataset)
        image, label = self.dataset[actual_idx]
        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image'].float()
            label = transformed['mask'].float()
            label = label.permute(2, 0, 1)
        return image, label

    def __len__(self):
        return len(self.dataset) * self.length_multiplier

In [7]:
train_transformation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    # A.RandomGamma (gamma_limit=(70, 130), always_apply=False, p=0.2),
    # A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

val_transformation = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std
        
    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor
    
unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

In [8]:
img_resize = (256, 256)
batch_size = 32
learning_rate = 1e-3
alpha = 0.5  # Weight of cross entropy loss 
num_epochs = 20
n_splits = 5

In [9]:
# Set random seed for reproducibility
from tqdm import tqdm
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# Initialize dataset
dataset = TrainDataset(
    img_dir=TRAIN_DIR,
    label_dir=TRAIN_MASK_DIR,
    resize=img_resize,
    transform=None
)

# K-Fold Cross-Validation
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f'\nFold {fold + 1}/{n_splits}')
    print('--------------------------------')

    # Subsets for the current fold
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    # Datasets with transformations
    train_dataset = AugmentDataset(train_subset, transform=train_transformation)
    val_dataset = AugmentDataset(val_subset, transform=val_transformation)

    # Data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    # Initialize model
    model = smp.UnetPlusPlus(
        encoder_name='resnet34',
        encoder_weights='imagenet',
        in_channels=3,
        classes=3
    )
    model = model.to(device)

    # Loss function
    criterion_ce = nn.CrossEntropyLoss()
    criterion_dice = smp.losses.DiceLoss(mode='multiclass')
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='min',                 # We want to minimize the validation loss
        factor=0.1,                 # Reduce LR by a factor of 0.5
        patience=1,                 # Number of epochs with no improvement after which LR will be reduced
        verbose=True,               # Print a message when LR is updated
        threshold=0.0001,           # Threshold for measuring the new optimum
        threshold_mode='rel',       # Mode for threshold
        cooldown=0,                 # Number of epochs to wait before resuming normal operation after LR has been reduced
        min_lr=1e-6                 # Lower bound on the learning rate
    )
    best_val_loss = 999
    # Training loop
    epoch_bar = tqdm(total=num_epochs, desc='Total Progress')
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch + 1}/{num_epochs}')
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(dim=1).long()
            optimizer.zero_grad()
            outputs = model(images)
            loss_ce = criterion_ce(outputs, labels)
            loss_dice = criterion_dice(outputs, labels)
            loss = alpha * loss_ce + (1 - alpha) * loss_dice
            loss.backward()
            optimizer.step()
           
            train_loss += loss.item() * images.size(0)

        train_epoch_loss = train_loss / len(train_loader.dataset)
        print(f'Training Loss: {train_epoch_loss:.4f}')

        # Validation loop
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                labels = labels.squeeze(dim=1).long()
                outputs = model(images)
                # loss_ce = criterion_ce(outputs, labels)
                loss_dice = criterion_dice(outputs, labels)
                # loss = alpha * loss_ce + (1 - alpha) * loss_dice
                val_loss += loss_dice.item() * images.size(0)
            val_epoch_loss = val_loss / len(val_loader.dataset)
            print(f'Validation Loss: {val_epoch_loss:.4f}')
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            checkpoint = { 
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'loss': val_loss,
            }
            model_path = f'model_fold_{fold + 1}.pth'
            torch.save(model.state_dict(), model_path)
        
        # Step the scheduler
        scheduler.step(val_epoch_loss)
        epoch_bar.update(1)
    epoch_bar.close()
    # Save the model for the current fold
    print(f'Model saved to {model_path}')


Fold 1/5
--------------------------------


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 87.1MB/s]
/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Total Progress:   0%|          | 0/20 [00:00<?, ?it/s]


Epoch 1/20
Training Loss: 0.7331


Total Progress:   5%|▌         | 1/20 [00:18<05:58, 18.88s/it]

Validation Loss: 0.6396

Epoch 2/20
Training Loss: 0.3627
Validation Loss: 0.4999


...

Validation Loss: 0.2406

Epoch 19/20
Training Loss: 0.1017


Total Progress:  95%|█████████▌| 19/20 [05:10<00:16, 16.24s/it]

Validation Loss: 0.2418

Epoch 20/20
Training Loss: 0.0994


Total Progress: 100%|██████████| 20/20 [05:26<00:00, 16.34s/it]

Validation Loss: 0.2398
Model saved to model_fold_5.pth


In [10]:
import torch
import segmentation_models_pytorch as smp

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# List to store models
models = []

# Number of folds
n_splits = 5  # Adjust based on your cross-validation

# Model parameters (should match those used during training)
model_params = {
    'encoder_name': 'resnet34',        # Same encoder as in training
    'encoder_weights': None,           # No pretraining since weights are loaded
    'in_channels': 3,
    'classes': 3                       # Number of segmentation classes
}

# Load models from each fold
for fold in range(n_splits):
    model = smp.UnetPlusPlus(**model_params)
    model_path = f'/kaggle/input/unetplusplus-resnet34/model_fold_{fold + 1}.pth'
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()  # Set model to evaluation mode
    models.append(model)

print(f'Loaded {len(models)} models for ensembling.')

/tmp/ipykernel_23/2267863117.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Loaded 5 models for ensembling.


In [11]:
test_dataset = TestDataset(img_dir="/kaggle/input/bkai-igh-neopolyp/test/test", resize=img_resize, transform=val_transformation)

In [12]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [13]:
color_dict= {0: (0, 0, 0),
             1: (255, 0, 0),
             2: (0, 255, 0)}
def mask_to_rgb(mask, color_dict):
    output = np.zeros((mask.shape[0], mask.shape[1], 3))

    for k in color_dict.keys():
        output[mask==k] = color_dict[k]

    return np.uint8(output)   

In [14]:
import cv2
import numpy as np

def Smoothed_img(image):
    resized_image = image
    smoothed_image = cv2.GaussianBlur(resized_image, (15, 15), 0)
    red_pixels = (smoothed_image[:, :, 2] > 100)
    green_pixels = (smoothed_image[:, :, 1] > 100)
    black_pixels = ~(red_pixels | green_pixels)
    smoothed_image[red_pixels] = [0, 0, 255]
    smoothed_image[green_pixels] = [0, 255, 0]
    smoothed_image[black_pixels] = [0, 0, 0]

    pixel_values = np.array(smoothed_image)
    # unique_values = np.unique(pixel_values)
    # print(unique_values)
    return smoothed_image

In [15]:
!mkdir prediction

In [16]:
with torch.no_grad():
    for image, h, w, name in test_loader:
        outputs_list = []
        for model in models:
            outputs = model(image.to(device))
            outputs = outputs.squeeze().cpu().numpy()
            outputs_list.append(outputs)
        
        # Average the class predictions
        avg_outputs = np.mean(outputs_list, axis=0)
        avg_outputs = np.argmax(avg_outputs, axis=0)
        avg_outputs = mask_to_rgb(avg_outputs, color_dict)
        
        w = w.item()
        h = h.item()
        new_size = (w, h)
        resized_image = cv2.resize(avg_outputs, new_size, interpolation=cv2.INTER_CUBIC)
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        resized_image = Smoothed_img(resized_image)
        cv2.imwrite(f"prediction/{name[0]}.png", resized_image)
        

In [17]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        # print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/kaggle/working/prediction'
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(r'output.csv', index=False)